In [1]:
import cv2
import numpy as np
import pickle

In [2]:
# Fungsi untuk memuat model KNN dan label mapping
def load_model():
    with open("save_model/knn_model.pkl", "rb") as f:
        knn_model_custom = pickle.load(f)
    return knn_model_custom

In [3]:
# Fungsi untuk mengklasifikasikan gambar tunggal
def classify_image(image_path, knn_model_custom):
    # Memuat model KNN dan label mapping
    knn_model = knn_model_custom["model"]
    label_mapping = knn_model_custom["label_mapping"]

    # Mengubah gambar menjadi fitur
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (32, 32))

    # Penghilangan background
    mask = np.zeros(resized_image.shape[:2], dtype=np.uint8)
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    masked_image = cv2.bitwise_and(resized_image, resized_image, mask=mask)

    # Ekstraksi fitur RGB
    rgb_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
    flattened_rgb = rgb_image.flatten()
    mean_rgb = np.mean(rgb_image, axis=(0, 1))
    mode_rgb = np.argmax(np.bincount(flattened_rgb))
    median_rgb = np.median(flattened_rgb)
    features_rgb = np.concatenate((flattened_rgb, mean_rgb, [mode_rgb], [median_rgb]))

    # Ekstraksi fitur HSV
    hsv_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)
    flattened_hsv = hsv_image.flatten()
    mean_hsv = np.mean(hsv_image, axis=(0, 1))
    mode_hsv = np.argmax(np.bincount(flattened_hsv))
    median_hsv = np.median(flattened_hsv)
    features_hsv = np.concatenate((flattened_hsv, mean_hsv, [mode_hsv], [median_hsv]))

    # Ekstraksi fitur Grayscale
    gray_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    flattened_gray = gray_image.flatten()
    mean_gray = np.mean(gray_image)
    mode_gray = np.argmax(np.bincount(flattened_gray))
    median_gray = np.median(flattened_gray)
    features_gray = np.concatenate((flattened_gray, [mean_gray], [mode_gray], [median_gray]))

    # Ekstraksi histogram
    hist = cv2.calcHist([masked_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist_flattened = hist.flatten()

    # Menggabungkan fitur RGB, HSV, Grayscale, dan histogram
    features = np.concatenate((features_rgb, features_hsv, features_gray, hist_flattened))

    # Melakukan prediksi
    label_index = knn_model.predict([features])[0]
    predicted_label = label_mapping[label_index]

    return predicted_label

In [4]:
# Memuat model KNN dan label mapping
knn_model_custom = load_model()

In [5]:
# Menentukan path gambar yang ingin diuji
image_path = "./../dataset_nitrogen_test/2/SWAPT1_042.jpg"

In [6]:
# Melakukan klasifikasi gambar
predicted_label = classify_image(image_path, knn_model_custom)

In [7]:
# Menampilkan hasil klasifikasi
print("Predicted Label:", predicted_label)

Predicted Label: 2
